# Resolve filenames to GUIDs

In order to address MediaFiles by GUIDs, we need to extract the GUID from the `SonyCiAsset.name` field.

## Steps

1. Get the list of assets from the DB
1. Filter out any name that does not start with `cpb-aacip-`
1. Split the name on any of:
   - `_` underscore
   - `.` period
   - `-dupe`

There are 8 records with `-dupe` in the name. All other records are correctly filtered with `_` and `.`.


In [3]:
from chowda.db import engine
from chowda.models import SonyCiAsset
from sqlmodel import Session, select
from re import search, split


def get_asset():
    with Session(engine) as session:
        statement = select(SonyCiAsset)
        results = session.exec(statement)

        return [
            (asset.id, split('_|\.|-dupe', asset.name)[0])
            for asset in results.all()
            if search('^cpb-aacip-', asset.name)
        ]


# assets = get_asset()


## Check the list

If needed, write the guid list to a file and check it manually.


In [4]:
# len(assets)


def write_assets():
    with open('guids.txt', 'w') as f:
        for asset in assets:
            f.write(f'{asset[1]}\n')

## Insert MediaFiles

Iterate through the list of assets:

- Split the GUID from the filename.
- Search the database for a matching MediaFile object.
  - Insert a new MediaFile object if it does not already exist.
- Find the SonyCiAsset object in the database.
- Link the MediaFile object to the SonyCiAsset object.


In [6]:
from chowda.models import MediaFile


def insert_media_files():
    with Session(engine) as session:
        for asset in assets:
            # Extract the GUID name
            guid = split(r'_|\.|-dupe', asset[1])[0]
            # media_file = session.get(MediaFile, guid)
            media_file = session.exec(
                select(MediaFile).where(MediaFile.guid == guid)
            ).first()
            if not media_file:
                # Create a new MediaFile with the new guid
                media_file = MediaFile(guid=guid)
                # session.add(media_file)
            ci_asset = session.get(SonyCiAsset, asset[0])
            # Add the asset to the existing MediaFile
            media_file.assets.append(ci_asset)
            # asset.media_files.append(media_file)
            session.add(media_file)
        session.commit()